In [10]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "Fish.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Question 1
# Reading the csv file
fish = pd.read_csv(file_content_stream)
fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [28]:
# Defining the target and input variables
X = fish[['Length1', 'Length2', 'Length3', 'Height', 'Width']]
Y = fish['Weight']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [29]:
# Performing LASSO as a variable selector
# Estimating lambda
lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

# Extracting optimal alpha
lasso_alpha = lasso_cv.alpha_

# Building model
lasso_md = Lasso(alpha = lasso_alpha, normalize = True).fit(X_train, Y_train)
lasso_md.coef_

array([21.64221829,  0.        , -0.        , 13.2691497 , 49.89296661])

In [13]:
# Dropping variables
X_train = X_train.drop(columns = ['Length2', 'Length3'], axis = 1)
X_test = X_test.drop(columns = ['Length2', 'Length3'], axis = 1)

In [14]:
def l2_normalization(X):
    
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [15]:
## Linear Regression
md1 = LinearRegression().fit(X_train, Y_train)

## Prediction on test dataset
md1_pred = md1.predict(X_test)

## Computing the mse
md1_mse = np.mean(np.power(md1_pred - Y_test, 2))
md1_mse

155056.501435243

In [24]:
# Ridge Regression
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
ridge_alpha = ridge_cv.alpha_

# Building the ridge model
ridge_md = Ridge(alpha = ridge_alpha).fit(X_train, Y_train)

# Predicting on the test dataset
md2_pred = ridge_md.predict(X_test)

# Computing the mse
md2_mse = np.mean(np.power(md2_pred, 2))
md2_mse

152032.18889048387

In [38]:
md1_mse - md2_mse

-71634.2466629879